<a href="https://colab.research.google.com/github/Michael26-lab/TRABAJO-FINAL/blob/main/PARTE%202%20Secci%C3%B3n%201%20orquestacion_pruebas_combinatorias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import itertools
import pandas as pd
import random



In [ ]:
# Parámetros del sistema a probar
parametros = {
    "browser": ["Chrome", "Firefox", "Edge"],
    "os": ["Windows", "Linux"],
    "auth": ["Correcta", "Incorrecta"],
    "network": ["Buena", "Lenta"]
}


In [ ]:
keys = parametros.keys()
values = parametros.values()

combinaciones = list(itertools.product(*values))
df_tests = pd.DataFrame(combinaciones, columns=keys)

df_tests


,browser,os,auth,network
0,Chrome,Windows,Correcta,Buena
1,Chrome,Windows,Correcta,Lenta
2,Chrome,Windows,Incorrecta,Buena
3,Chrome,Windows,Incorrecta,Lenta
4,Chrome,Linux,Correcta,Buena
5,Chrome,Linux,Correcta,Lenta
6,Chrome,Linux,Incorrecta,Buena
7,Chrome,Linux,Incorrecta,Lenta
8,Firefox,Windows,Correcta,Buena
9,Firefox,Windows,Correcta,Lenta


In [ ]:
# Simulación de historial de ejecuciones previas
df_tests["fallos_previos"] = [
    random.randint(0, 5) for _ in range(len(df_tests))
]

df_tests.head()


,browser,os,auth,network,fallos_previos
0,Chrome,Windows,Correcta,Buena,3
1,Chrome,Windows,Correcta,Lenta,5
2,Chrome,Windows,Incorrecta,Buena,3
3,Chrome,Windows,Incorrecta,Lenta,1
4,Chrome,Linux,Correcta,Buena,1


In [ ]:
# Cálculo simple de riesgo
df_tests["riesgo"] = df_tests["fallos_previos"] / df_tests["fallos_previos"].max()


In [ ]:
df_priorizados = df_tests.sort_values(by="riesgo", ascending=False)
df_priorizados.head(10)


,browser,os,auth,network,fallos_previos,riesgo
1,Chrome,Windows,Correcta,Lenta,5,1.0
7,Chrome,Linux,Incorrecta,Lenta,5,1.0
6,Chrome,Linux,Incorrecta,Buena,5,1.0
14,Firefox,Linux,Incorrecta,Buena,5,1.0
15,Firefox,Linux,Incorrecta,Lenta,5,1.0
23,Edge,Linux,Incorrecta,Lenta,5,1.0
8,Firefox,Windows,Correcta,Buena,4,0.8
11,Firefox,Windows,Incorrecta,Lenta,4,0.8
18,Edge,Windows,Incorrecta,Buena,4,0.8
22,Edge,Linux,Incorrecta,Buena,4,0.8


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder


In [ ]:
# Codificación
encoder = LabelEncoder()
X = df_tests.drop(columns=["fallos_previos", "riesgo"]).apply(encoder.fit_transform)
y = (df_tests["fallos_previos"] > 2).astype(int)


In [ ]:
modelo = LogisticRegression()
modelo.fit(X, y)

df_tests["probabilidad_fallo"] = modelo.predict_proba(X)[:,1]


In [ ]:
df_tests.sort_values(by="probabilidad_fallo", ascending=False).head(10)


,browser,os,auth,network,fallos_previos,riesgo,probabilidad_fallo
2,Chrome,Windows,Incorrecta,Buena,3,0.6,0.857199
18,Edge,Windows,Incorrecta,Buena,4,0.8,0.857193
10,Firefox,Windows,Incorrecta,Buena,4,0.8,0.857187
6,Chrome,Linux,Incorrecta,Buena,5,1.0,0.827633
22,Edge,Linux,Incorrecta,Buena,4,0.8,0.827625
14,Firefox,Linux,Incorrecta,Buena,5,1.0,0.827618
3,Chrome,Windows,Incorrecta,Lenta,1,0.2,0.667113
0,Chrome,Windows,Correcta,Buena,3,0.6,0.667111
19,Edge,Windows,Incorrecta,Lenta,0,0.0,0.667102
16,Edge,Windows,Correcta,Buena,3,0.6,0.667099
